In [1]:
import os
import cv2
import numpy as np

#Encoding and Split data into Train/Test Sets
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

#Tensorflow Keras CNN Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras import callbacks

#Plot Images
import matplotlib.pyplot as plt



2022-11-29 13:56:07.041635: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Generate Training and Testing Data

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   rotation_range=45,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split = .2)
test_datagen = ImageDataGenerator(rescale=1./255,
                                  validation_split = .2)


In [3]:
epochs= 64
batch_size= 32 
size= 150 

## Split critter_data into training, testing and validation data 

In [ ]:
import splitfolders

# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio("input_folder", output="output",
    seed=12, ratio=(.8, .1, .1), group_prefix=None, move=False) # default values



## Get training and testing data

In [4]:
train_generator = train_datagen.flow_from_directory(
    directory=r"./critter_data",
    target_size=(size, size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    subset="training",
    seed=12)

valid_generator = train_datagen.flow_from_directory('./critter_data',
                                                    target_size=(size,size),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    subset='validation', 
                                                    color_mode="grayscale", 
                                                    seed= 12, 
                                                   shuffle=True)

Found 3156 images belonging to 5 classes.


In [5]:


test_generator = test_datagen.flow_from_directory('./critter_data',
                                                  target_size=(size,size),
                                                  batch_size=1,
                                                  class_mode='categorical',
                                                  shuffle = False,
                                                  subset='validation', 
                                                  color_mode="grayscale", 
                                                  seed = 12)

Found 2526 images belonging to 5 classes.
Found 630 images belonging to 5 classes.


## Architecture of CNN

In [6]:
# Use a sequential model because each layer has one input tensor (np.array) and one output tensor 
model = Sequential()

# implement simple Convolutional Neural Network (CNN) with Conv2D and 64 filters 
# kernel_size specifying the width and height of the 2D convolution window
# chose 3 beacause input images size = 128 
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu', input_shape = (size,size,1)))

# filters: Layers early in the network architecture (i.e., closer to the actual input image) 
# learn fewer convolutional filters while layers deeper in the network 
# (i.e., closer to the output predictions) will learn more filters.

# Max pooling is used to reduce the spatial dimensions of the output volume
model.add(MaxPooling2D(pool_size=(2,2)))

# padding = same so that the output volume size matches the input volume size
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))

model.add(MaxPooling2D(pool_size=(2,2)))

# converting multi-dimensional array into one dimensional flatten array or say single dimensional array
model.add(Flatten())

# Dense Layer is used to classify image based on output from convolutional layers.
model.add(Dense(128, activation='relu'))
# Each Layer in the Neural Network contains neurons, which compute the weighted average of its input and this weighted average is passed through a non-linear function, called as an “activation function”.
# in this case, activation function = relu 
model.add(Dense(64, activation='relu'))

# randomly selected neurons (50%) are ignored during training. They are “dropped out” randomly
# (prevent overfitting on the training data)
model.add(Dropout(rate=0.5))

#Softmax is an activation function that scales numbers/logits into probabilities. The output of a Softmax is a vector (say v ) with probabilities of each possible outcome. 
model.add(Dense(5, activation = "softmax"))



2022-11-29 13:56:32.266246: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Compile the model

In [8]:
# use adam optimizer algorithm for optimization technique for gradient descent
# loss= cat_cross: Used as a loss function for multi-class classification model where there are two or more output labels
# metrics= accuracy: Calculates how often predictions equal labels
model.compile(optimizer=Adam(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])


In [9]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size


### Add Early Stop to optimize time 

In [10]:
# Stop training when a monitored metric has stopped improving
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 5, 
                                        restore_best_weights = True)

### Add checkpoints to be able to pause training 

In [ ]:

checkpoint_path = "model_ignore/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


# Create a callback that saves the model's weights
cp_callback = tensorflow.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

## Fit model 

In [11]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    callbacks= [earlystopping, cp_callback],
                    epochs=epochs)

/opt/anaconda3/envs/Ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/64
98/98 [==============================] - 2095s 21s/step - loss: 1.6982 - accuracy: 0.2916 - val_loss: 1.4417 - val_accuracy: 0.3506
Epoch 2/64
98/98 [==============================] - 2120s 22s/step - loss: 1.4805 - accuracy: 0.3367 - val_loss: 1.4537 - val_accuracy: 0.3446
Epoch 3/64
98/98 [==============================] - 2237s 23s/step - loss: 1.4620 - accuracy: 0.3380 - val_loss: 1.4104 - val_accuracy: 0.3646
Epoch 4/64
98/98 [==============================] - 2080s 21s/step - loss: 1.4444 - accuracy: 0.3646 - val_loss: 1.4142 - val_accuracy: 0.3658
Epoch 5/64
98/98 [==============================] - 1967s 20s/step - loss: 1.4449 - accuracy: 0.3531 - val_loss: 1.4056 - val_accuracy: 0.3654
Epoch 6/64
98/98 [==============================] - 2756s 28s/step - loss: 1.4388 - accuracy: 0.3582 - val_loss: 1.4057 - val_accuracy: 0.3622
Epoch 7/64
98/98 [==============================] - 3731s 38s/step - loss: 1.4434 - accuracy: 0.3569 - val_loss: 1.4088 - val_accuracy: 0.3766

KeyboardInterrupt: 

## Evaluate the Model 

In [ ]:
model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_VALID)

## OR Load Weights of previously built model 

In [ ]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

### Predict the Output

In [ ]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

### ALWAYS reset test_generator before whenever you call the predict_generator 

In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)
